# LangChain Multi-Query for RAG

## Introduction
One of the advanced features in LangChain is Multi-Query for RAG, which allows for more efficient and effective information retrieval from large datasets.

## What is Multi-Query for RAG?

Multi-Query for RAG is a technique used to enhance the retrieval process in retrieval-augmented generation systems. It involves generating multiple queries from a single input and using these queries to retrieve a more diverse and relevant set of documents from the database. This approach improves the quality of the retrieved information, leading to better and more accurate generated responses.

## Key Components

1. **Query Generation**: Instead of generating a single query, the system creates multiple queries based on different aspects or perspectives of the input. These queries can be variations or expansions of the original input.

2. **Document Retrieval**: Each generated query is used to search the indexed documents. The results from all queries are combined to form a more comprehensive set of retrieved documents.

3. **Response Generation**: The retrieved documents are then used to generate the final response. By incorporating information from multiple queries, the generated response is more likely to be accurate and relevant.

## Benefits of Multi-Query for RAG

- **Improved Diversity**: Multiple queries increase the chances of retrieving diverse pieces of information, covering different aspects of the input topic.
- **Enhanced Relevance**: Combining results from multiple queries can lead to a more relevant and contextually appropriate set of documents.
- **Robustness**: Multi-query retrieval is more robust to variations and ambiguities in the input, as different queries can capture different interpretations.

## Example Workflow

1. **Input Processing**: The user provides an input query.
2. **Query Generation**: The system generates multiple queries from the input.
3. **Document Retrieval**: Each query is used to search the document index, and the results are combined.
4. **Response Generation**: The retrieved documents are used by the language model to generate the final response.

<div>
<img src="https://education-team-2020.s3.eu-west-1.amazonaws.com/ai-eng/images-langchain-4/muti.avif" alt='auto' width="1000"/>
</div>

## Example Code

Here is a simplified example of how Multi-Query for RAG might be implemented using LangChain:

```python
from langchain.chains import MultiQueryRAGChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.indexes import VectorIndex

# Define the base prompt template
base_prompt_template = PromptTemplate(
    input_variables=["input_text"],
    template="Generate multiple queries from the following input: {input_text}"
)

# Initialize the language model
llm = OpenAI(model="text-davinci-003")

# Create the Multi-Query RAG Chain
multi_query_chain = MultiQueryRAGChain(
    prompt=base_prompt_template,
    llm=llm,
    index=VectorIndex()  # Assuming VectorIndex is already populated with documents
)

# Input text
input_text = "Tell me about the history of artificial intelligence."

# Run the chain and get the response
response = multi_query_chain.run({"input_text": input_text})
print(response)


## Getting Data

We will download an existing dataset from Hugging Face Datasets.

In [ ]:
!pip install datasets langchain_community

In [ ]:
from datasets import load_dataset

data = load_dataset("jamescalam/ai-arxiv-chunked", split="train")
data

In [4]:
from langchain.docstore.document import Document

docs = []

for row in data:
    doc = Document(
        page_content=row["chunk"],
        metadata={
            "title": row["title"],
            "source": row["source"],
            "id": row["id"],
            "chunk-id": row["chunk-id"],
            "text": row["chunk"]
        }
    )
    docs.append(doc)

In [ ]:
from dotenv import load_dotenv, find_dotenv
import os
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.getenv("PINECONE_API_KEY") or "YOUR_API_KEY"

## Embedding and Vector DB Setup

Initialize our embedding model:

In [8]:
import os
from getpass import getpass
from langchain.embeddings.openai import OpenAIEmbeddings

model_name = "text-embedding-ada-002"

embed = OpenAIEmbeddings(
    model=model_name, openai_api_key=OPENAI_API_KEY, disallowed_special=()
)

<ipython-input-8-d00f2f1001bc>:7: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embed = OpenAIEmbeddings(


Now we create our vector DB to store our vectors. For this we need to get a [free Pinecone API key](https://app.pinecone.io) — the API key can be found in the "API Keys" button found in the left navbar of the Pinecone dashboard.

In [ ]:
!pip install -qU langchain-pinecone pinecone-notebooks

In [13]:
from pinecone import Pinecone
# configure client
pc = Pinecone(api_key=api_key)

Now we setup our index specification, this allows us to define the cloud provider and region where we want to deploy our index. You can find a list of all [available providers and regions here](https://docs.pinecone.io/docs/projects).

In [14]:
from pinecone import ServerlessSpec

spec = ServerlessSpec(
    cloud="aws", region="us-east-1"
)

Creating an index, we set `dimension` equal to to dimensionality of Ada-002 (`1536`), and use a `metric` also compatible with Ada-002 (this can be either `cosine` or `dotproduct`). We also pass our `spec` to index initialization.

In [15]:
import time

index_name = "langchain-multi-query-demo"
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# check if index already exists (it shouldn't if this is first time)
if index_name not in existing_indexes:
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=1536,  # dimensionality of ada 002
        metric='dotproduct',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

Populate our index:

In [16]:
len(docs)

41584

In [17]:
# if you want to speed things up to follow along
#docs = docs[:5000]

In [ ]:
!pip install tiktoken

In [23]:
from tqdm.auto import tqdm

batch_size = 100

for i in tqdm(range(0, len(docs), batch_size)):
    i_end = min(len(docs), i+batch_size)
    docs_batch = docs[i:i_end]
    # get IDs
    ids = [f"{doc.metadata['id']}-{doc.metadata['chunk-id']}" for doc in docs_batch]
    # get text and embed
    texts = [d.page_content for d in docs_batch]
    embeds = embed.embed_documents(texts=texts)
    # get metadata
    metadata = [d.metadata for d in docs_batch]
    to_upsert = zip(ids, embeds, metadata)
    index.upsert(vectors=to_upsert)

  0%|          | 0/416 [00:00<?, ?it/s]

## Multi-Query with LangChain

Now we switch across to using our populated index as a vectorstore in Langchain.

In [24]:
from langchain.vectorstores import Pinecone

text_field = "text"

vectorstore = Pinecone(index, embed.embed_query, text_field)

<ipython-input-24-8c5380700a91>:5: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.18 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-pinecone package and should be used instead. To use it run `pip install -U :class:`~langchain-pinecone` and import as `from :class:`~langchain_pinecone import Pinecone``.
  vectorstore = Pinecone(index, embed.embed_query, text_field)
/usr/local/lib/python3.11/dist-packages/langchain_community/vectorstores/pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [25]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

<ipython-input-25-7e20abb53403>:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)


We initialize the `MultiQueryRetriever`:

In [26]:
from langchain.retrievers.multi_query import MultiQueryRetriever

retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(), llm=llm
)

We set logging so that we can see the queries as they're generated by our LLM.

In [27]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

To query with our multi-query retriever we call the `get_relevant_documents` method.

In [28]:
question = "tell me about llama 2?"

docs = retriever.get_relevant_documents(query=question)
len(docs)

<ipython-input-28-c06808f0ce89>:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query=question)
INFO:langchain.retrievers.multi_query:Generated queries: ['What information can you provide about llama 2?', 'What are some details about llama 2?', 'Can you share some insights on llama 2?']


8

From this we get a variety of docs retrieved by each of our queries independently. By default the `retriever` is returning `3` docs for each query — totalling `9` documents — however, as there is some overlap we actually return `6` unique docs.

In [29]:
docs

[Document(metadata={'chunk-id': '1', 'id': '2307.09288', 'source': 'http://arxiv.org/pdf/2307.09288', 'title': 'Llama 2: Open Foundation and Fine-Tuned Chat Models'}, page_content='Alan Schelten Ruan Silva Eric Michael Smith Ranjan Subramanian Xiaoqing Ellen Tan Binh Tang\nRoss Taylor Adina Williams Jian Xiang Kuan Puxin Xu Zheng Yan Iliyan Zarov Yuchen Zhang\nAngela Fan Melanie Kambadur Sharan Narang Aurelien Rodriguez Robert Stojnic\nSergey Edunov Thomas Scialom\x03\nGenAI, Meta\nAbstract\nIn this work, we develop and release Llama 2, a collection of pretrained and ﬁne-tuned\nlarge language models (LLMs) ranging in scale from 7 billion to 70 billion parameters.\nOur ﬁne-tuned LLMs, called L/l.sc/a.sc/m.sc/a.sc /two.taboldstyle-C/h.sc/a.sc/t.sc , are optimized for dialogue use cases. Our\nmodels outperform open-source chat models on most benchmarks we tested, and based on\nourhumanevaluationsforhelpfulnessandsafety,maybeasuitablesubstituteforclosedsource models. We provide a detailed 

## Adding the Generation in RAG

So far we've built a multi-query powered **R**etrieval **A**ugmentation chain. Now, we need to add **G**eneration.

In [30]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

QA_PROMPT = PromptTemplate(
    input_variables=["query", "contexts"],
    template="""You are a helpful assistant who answers user queries using the
    contexts provided. If the question cannot be answered using the information
    provided say "I don't know".

    Contexts:
    {contexts}

    Question: {query}""",
)

# Chain
qa_chain = LLMChain(llm=llm, prompt=QA_PROMPT)

<ipython-input-30-ccfccfd854d8>:17: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  qa_chain = LLMChain(llm=llm, prompt=QA_PROMPT)


In [31]:
out = qa_chain(
    inputs={
        "query": question,
        "contexts": "\n---\n".join([d.page_content for d in docs])
    }
)
out["text"]

<ipython-input-31-d45b0d3d2a21>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  out = qa_chain(


'Llama 2 is a collection of pretrained and fine-tuned large language models (LLMs) ranging from 7 billion to 70 billion parameters. The fine-tuned LLMs in Llama 2, called L/l.sc/a.sc/m.sc/a.sc/two.taboldstyle-C/h.sc/a.sc/t.sc, are optimized for dialogue use cases. These models outperform open-source chat models on most benchmarks tested and may be a suitable substitute for closed-source models based on humane evaluations for helpfulness and safety. The approach to fine-tuning and safety is detailed in the work.'

## Chaining Everything with a SequentialChain

We can pull together the logic above into a function or set of methods, whatever is prefered — however if we'd like to use LangChain's approach to this we must "chain" together multiple chains. The first retrieval component is (1) not a chain per se, and (2) requires processing of the output. To do that, and fit with LangChain's "chaining chains" approach, we setup the _retrieval_ component within a `TransformChain`:

In [32]:
from langchain.chains import TransformChain

def retrieval_transform(inputs: dict) -> dict:
    docs = retriever.get_relevant_documents(query=inputs["question"])
    docs = [d.page_content for d in docs]
    docs_dict = {
        "query": inputs["question"],
        "contexts": "\n---\n".join(docs)
    }
    return docs_dict

retrieval_chain = TransformChain(
    input_variables=["question"],
    output_variables=["query", "contexts"],
    transform=retrieval_transform
)

Now we chain this with our generation step using the `SequentialChain`:

In [33]:
from langchain.chains import SequentialChain

rag_chain = SequentialChain(
    chains=[retrieval_chain, qa_chain],
    input_variables=["question"],  # we need to name differently to output "query"
    output_variables=["query", "contexts", "text"]
)

Then we perform the full RAG pipeline:

In [34]:
out = rag_chain({"question": question})
out["text"]

INFO:langchain.retrievers.multi_query:Generated queries: ['What information can you provide about llama 2?', 'What are some details about llama 2?', 'Can you share some insights on llama 2?']


'Llama 2 is a collection of pretrained and fine-tuned large language models (LLMs) developed by the team mentioned in the provided context. These models range in scale from 7 billion to 70 billion parameters and are optimized for dialogue use cases. The fine-tuned LLMs, such as L/l.sc/a.sc/m.sc/a.sc/two.taboldstyle-C/h.sc/a.sc/t.sc, outperform open-source chat models on various benchmarks and may serve as a suitable substitute for closed-source models based on humane evaluations for helpfulness and safety. The approach to fine-tuning and safety is detailed in their work.'

---

## Custom Multiquery

We'll try this with two prompts, both encourage more variety in search queries.

**Prompt A**
```
Your task is to generate 3 different search queries that aim to
answer the user question from multiple perspectives.
Each query MUST tackle the question from a different viewpoint,
we want to get a variety of RELEVANT search results.
Provide these alternative questions separated by newlines.
Original question: {question}
```


**Prompt B**
```
Your task is to generate 3 different search queries that aim to
answer the user question from multiple perspectives. The user questions
are focused on Large Language Models, Machine Learning, and related
disciplines.
Each query MUST tackle the question from a different viewpoint, we
want to get a variety of RELEVANT search results.
Provide these alternative questions separated by newlines.
Original question: {question}
```

In [65]:
from typing import List
from langchain.chains import LLMChain
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser


# Output parser will split the LLM result into a list of queries
class LineList(BaseModel):
    # "lines" is the key (attribute name) of the parsed output
    lines: List[str] = Field(description="Lines of text")


class LineListOutputParser(PydanticOutputParser):
    def __init__(self) -> None:
        super().__init__(pydantic_object=LineList)

    def parse(self, text: str) -> LineList:
        lines = text.strip().split("\n")
        return LineList(lines=lines)


output_parser = LineListOutputParser()

template = """
Your task is to generate 3 different search queries that aim to
answer the user question from multiple perspectives. The user questions
are focused on Large Language Models, Machine Learning, and related
disciplines.
Each query MUST tackle the question from a different viewpoint, we
want to get a variety of RELEVANT search results.
Provide these alternative questions separated by newlines.
Original question: {question}
"""

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template=template,
)
llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

# Chain
llm_chain = LLMChain(llm=llm, prompt=QUERY_PROMPT, output_parser=output_parser)

In [72]:
from typing import List

from langchain_core.output_parsers import BaseOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field


# Output parser will split the LLM result into a list of queries
class LineListOutputParser(BaseOutputParser[List[str]]):
    """Output parser for a list of lines."""

    def parse(self, text: str) -> List[str]:
        lines = text.strip().split("\n")
        return list(filter(None, lines))  # Remove empty lines


output_parser = LineListOutputParser()

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template = """
    Your task is to generate 3 different search queries that aim to
    answer the user question from multiple perspectives. The user questions
    are focused on Large Language Models, Machine Learning, and related
    disciplines.
    Each query MUST tackle the question from a different viewpoint, we
    want to get a variety of RELEVANT search results.
    Provide these alternative questions separated by newlines.
    Original question: {question}
    """,
)
llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

# Chain
llm_chain = QUERY_PROMPT | llm | output_parser

# Other inputs
question = "What are the key features and capabilities of Large Language Model Llama 2?"

In [73]:
# Run
retriever = MultiQueryRetriever(
    retriever=vectorstore.as_retriever(), llm_chain=llm_chain, parser_key="lines"
)  # "lines" is the key (attribute name) of the parsed output

# Results
unique_docs = retriever.invoke("How does Llama 2 compare to other Large Language Models in terms of performance and efficiency?")
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What are the key performance metrics used to evaluate Large Language Models like Llama 2, and how does it stack up against other models in terms of these metrics?', '2. How do different Large Language Models, including Llama 2, compare in terms of computational efficiency and resource utilization?', '3. What are the advantages and disadvantages of Llama 2 compared to other Large Language Models when it comes to handling complex language tasks and generating high-quality text outputs?']


9

In [75]:
unique_docs

[Document(metadata={'chunk-id': '14', 'id': '2206.04615', 'source': 'http://arxiv.org/pdf/2206.04615', 'title': 'Beyond the Imitation Game: Quantifying and extrapolating the capabilities of language models'}, page_content='2\n3.4.3 Even programmatic measures of model capability can be highly subjective . . . . . . . 15\n3.5 Even large language models are brittle . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 15\n3.6 Social bias in large language models . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 17\n3.7 Performance on non-English languages . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 20\n4 Behavior on selected tasks 21\n4.1 Checkmate-in-one task . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 22\n4.2 Periodic elements task . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 23\n5 Additional related work 24\n6 Discussion 25'),
 Document(metadata={'chunk-id': '10', 'id': '2304.

Putting this together in another `SequentialChain`:

In [76]:
retrieval_chain = TransformChain(
    input_variables=["question"],
    output_variables=["query", "contexts"],
    transform=retrieval_transform
)

rag_chain = SequentialChain(
    chains=[retrieval_chain, qa_chain],
    input_variables=["question"],  # we need to name differently to output "query"
    output_variables=["query", "contexts", "text"]
)

And asking again:

In [77]:
out = rag_chain({"question": question})
out["text"]

INFO:langchain.retrievers.multi_query:Generated queries: ['1. How does Large Language Model Llama 2 compare to other similar models in terms of performance and efficiency?', '2. What are the potential applications and use cases of Large Language Model Llama 2 in natural language processing and artificial intelligence?', '3. What are the latest advancements and updates in Large Language Model Llama 2, and how do they enhance its capabilities and functionalities?']


'The key features and capabilities of Large Language Model Llama 2 include being a collection of pretrained and fine-tuned models ranging from 7 billion to 70 billion parameters. The fine-tuned Llama 2 models, optimized for dialogue use cases, outperform open-source chat models on most benchmarks tested. They are also considered to be a suitable substitute for closed-source models based on humane evaluations for helpfulness and safety.'

After finishing, delete your Pinecone index to save resources:

In [78]:
pc.delete_index(index_name)

---